In [63]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti

In [64]:
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_mixed_effects/age-stim-interaction_pd-vs-ad'
clinical_data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_analyses/ses-01/sub-all/ad_pd_mixed_data/ad_pd_full_data.csv'
# clin_path = 'path to clinical values'
print('Will save to: ', out_dir)
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

Will save to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_mixed_effects/age-stim-interaction_pd-vs-ad


# Import Clinical Data

In [65]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clinical_data_path)
    
# #Remove subjects
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

data_df

,"Patient # CDR, ADAS",Age,% Change from baseline (ADAS-Cog11),Subiculum Connectivity,Subiculum Grey Matter,Subiculum White Matter,Subiculum CSF,Subiculum Total,Disease,Standardized Age,Standardized Percent Improvement,Standardized Subiculum Connectivity,Standardized Subiculum Grey Matter,Standardized Subiculum White Matter,Standardized Subiculum CSF,Standardized Subiculum Total,One Hot Disease
0,101,62,-21.428571,56.864683,1.646994,0.510111,2.975675,1.280978,Alzheimer,0.289341,0.289341,-1.179578,1.970962,0.663981,1.773701,1.789087,1
1,102,77,-36.363636,52.970984,-0.865158,-1.615736,0.740780,-1.560273,Alzheimer,-0.016378,-0.016378,-1.635523,-0.125938,-1.611307,-0.272553,-1.093456,1
2,103,76,-78.947368,62.459631,0.319460,-0.796399,0.532453,-0.341032,Alzheimer,-0.888064,-0.888064,-0.524419,0.862866,-0.734373,-0.463295,0.143505,1
3,104,65,-129.411765,59.611631,-0.440643,-0.159752,0.405263,-0.358042,Alzheimer,-1.921065,-1.921065,-0.857915,0.228406,-0.052972,-0.579749,0.126248,1
4,105,50,-10.526316,57.928350,-0.328427,-0.229875,-0.988805,-0.339817,Alzheimer,0.512509,0.512509,-1.055024,0.322072,-0.128024,-1.856148,0.144737,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,30,58,-1.388889,18.674670,6.973009,2.352260,6.973009,0.872723,Parkinson,-0.005890,-0.005890,-0.628666,0.291196,1.089313,0.291196,-0.792728,0
68,31,64,1.449275,15.353030,-8.205764,5.376292,-8.205764,8.959061,Parkinson,1.034438,1.034438,-1.099623,-3.303201,2.909124,-3.303201,2.775495,0
69,33,60,-1.398601,15.050219,9.086053,0.302823,9.086053,1.435556,Parkinson,-0.009451,-0.009451,-1.142557,0.791574,-0.144002,0.791574,-0.544369,0
70,36,52,-2.797203,17.382020,8.416993,1.409619,8.416993,-0.365509,Parkinson,-0.522107,-0.522107,-0.811944,0.633137,0.522049,0.633137,-1.339117,0


Get More Clinical Data As Needed

In [66]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = ['Standardized Age', 'One Hot Disease']
outcome_column = 'Standardized Percent Improvement'

#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1 = data_df[clinical_information_column]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
clinical_df_1['outcome'] = data_df.loc[:, [outcome_column]]

# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
display(clinical_df_1)

,Standardized Age,One Hot Disease,outcome
subject_id,,,
101,0.289341,1,0.289341
102,-0.016378,1,-0.016378
103,-0.888064,1,-0.888064
104,-1.921065,1,-1.921065
105,0.512509,1,0.512509
...,...,...,...
30,-0.005890,0,-0.005890
31,1.034438,0,1.034438
33,-0.009451,0,-0.009451


Import Imaging Data

In [67]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='*_T*.nii*')

#Standardize matrix via z score
prepped_matrix = pd.DataFrame()
from calvin_utils.statistical_utils.z_score_matrix import z_score_matrix
for column in matrix.columns:
    #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
    print(column)
    new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA
    #Standardize column by column
    prepped_matrix[new_name] = z_score_matrix(matrix[column])
    #Set values outside brain back to zero
    prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
prepped_matrix = prepped_matrix.transpose()
prepped_matrix['subject_id'] = [str(col).split('_')[0] for col in prepped_matrix.index]
prepped_matrix.set_index('subject_id', inplace=True)
neuroimaging_df_1 = prepped_matrix
neuroimaging_df_1.index = neuroimaging_df_1.index.astype(str)
#Display results
display(neuroimaging_df_1)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*_T*.nii*
116_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  195.56159244766604 (902629,)
post z score max:  36.16030539421581 (902629,)
1203_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  162.6742436890163 (902629,)
post z score max:  28.965697798110018 (902629,)
139_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  193.79302616660425 (902629,)
post z score max:  33.171095565585055 (902629,)
147_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  209.41335490

,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Import More Imaging Data

In [68]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='*T.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------
#Standardize matrix via z score
# neuroimaging_df_2 = pd.DataFrame()
# from calvin_utils.z_score_matrix import z_score_matrix
# if performed_z_score:
#     for column in matrix.columns:
#         #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
#         new_name = int(column.split('_')[0])
#         #Standardize column by column
#         prepped_matrix[new_name] = z_score_matrix(matrix[column])
#         #Set values outside brain back to zero
#         prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0].split('MDST')[1] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
display(neuroimaging_df_2)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity/*T.nii*


,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# perform masking

In [69]:
#Mask Dataframes
mask_path = None #'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)


Dataframes have been masked such that their shapes are:  (50, 225222)
Dataframes have been masked such that their shapes are:  (32, 225222)


# Perform Standardization

In [70]:
# Define a lambda function for standardization
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
neuroimaging_df_1.iloc[:, 0:] = scaler.fit_transform(neuroimaging_df_1.iloc[:, 0:])
neuroimaging_df_2.iloc[:, 0:] = scaler.fit_transform(neuroimaging_df_2.iloc[:, 0:])

In [71]:
neuroimaging_df_1

,93491,93492,93493,93582,93583,93672,93673,93674,93764,102954,...,798109,798110,798190,798191,798192,798193,798281,798282,798283,798373
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.072526,0.028231,0.137057,0.067111,0.090728,0.121536,0.115671,0.187223,0.196694,0.185268,...,0.832366,0.780728,-0.120228,-0.070138,-0.081724,-0.088126,-0.109185,-0.072590,-0.091481,-0.131085
1203,0.760362,0.477588,0.152470,0.980367,0.771788,0.956861,0.970555,0.878367,0.873298,-0.907712,...,0.635405,0.633537,1.123429,1.212856,1.276081,1.116667,1.152950,1.260941,1.308556,1.246259
139,1.165434,1.089394,1.131826,0.973582,0.951851,0.923947,0.831367,0.889927,0.749248,0.128274,...,0.993769,0.959919,0.178940,0.358247,0.596250,0.822984,0.213267,0.414414,0.635269,0.440591
147,2.011486,1.792369,1.487507,1.820099,1.692860,1.584822,1.527240,1.490088,1.261155,0.166600,...,1.233881,1.192355,0.969693,1.151523,1.386193,1.437560,1.032846,1.268186,1.506246,1.369563
135,-0.661065,-0.661364,-0.482411,-0.583445,-0.587504,-0.271633,-0.414294,-0.479556,-0.216181,0.011714,...,0.727218,0.639360,-0.019674,-0.133375,-0.373121,-0.597730,-0.039164,-0.169586,-0.389523,-0.222028
101,0.565606,0.619741,0.824664,0.289703,0.463782,0.107352,0.211271,0.521104,0.243131,0.806286,...,1.397196,1.314386,-0.642376,-0.477894,-0.313008,-0.107899,-0.644884,-0.488569,-0.393913,-0.643514
150,0.330519,0.381154,0.142842,0.679395,0.610988,0.646290,0.740776,0.610532,0.694649,-1.119133,...,-0.926785,-0.687593,1.192138,1.217543,1.262101,1.182973,1.162818,1.188827,1.219496,1.203663
122,0.813956,0.736161,0.672523,0.837142,0.810767,0.790341,0.772754,0.803769,0.694188,-0.554856,...,0.628986,0.721795,0.560733,0.658505,0.799889,0.892512,0.578615,0.690249,0.805865,0.708647
1202,0.548826,0.616749,0.836198,0.250414,0.463559,0.093901,0.240913,0.533392,0.408220,1.372528,...,1.013976,1.214979,-0.717000,-0.569658,-0.436120,-0.189897,-0.752036,-0.612510,-0.503299,-0.753422


## Join Imaging Data to the Clinical Data

In [72]:
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
# Stack neuroimaging dataframes
stacked_neuroimaging_df = pd.concat([neuroimaging_df_1, neuroimaging_df_2], axis=0)

# Perform inner join with clinical_df
merged_df = preprocess_colnames_for_regression(clinical_df_1.join(stacked_neuroimaging_df, how='inner'))
merged_df

,Standardized_Age,One_Hot_Disease,outcome,var_93491,var_93492,var_93493,var_93582,var_93583,var_93672,var_93673,...,var_798109,var_798110,var_798190,var_798191,var_798192,var_798193,var_798281,var_798282,var_798283,var_798373
subject_id,,,,,,,,,,,,,,,,,,,,,
101,0.289341,1,0.289341,0.565606,0.619741,0.824664,0.289703,0.463782,0.107352,0.211271,...,1.397196,1.314386,-0.642376,-0.477894,-0.313008,-0.107899,-0.644884,-0.488569,-0.393913,-0.643514
102,-0.016378,1,-0.016378,0.569502,0.675135,0.978507,0.285238,0.542427,0.006029,0.135294,...,1.697887,1.731429,-0.680607,-0.529925,-0.293708,0.044063,-0.712084,-0.546895,-0.413581,-0.679440
103,-0.888064,1,-0.888064,1.260100,1.192252,1.231898,0.953457,0.973259,0.772010,0.757829,...,1.321081,1.168334,-0.003070,0.211064,0.451404,0.654128,0.044497,0.286226,0.510745,0.284952
104,-1.921065,1,-1.921065,-0.108620,-0.267096,-0.140600,-0.299919,-0.297932,-0.393635,-0.302277,...,0.372969,0.483971,-1.020602,-0.942247,-0.909154,-0.815275,-0.985672,-0.934981,-0.893335,-0.986823
105,0.512509,1,0.512509,-2.487030,-2.735946,-2.768873,-2.984791,-3.040821,-3.092803,-3.136508,...,-1.579894,-2.119148,-2.675310,-2.644539,-2.457218,-2.322484,-2.713788,-2.650655,-2.487042,-2.616541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,-0.005890,0,-0.005890,-0.907234,-1.056045,-1.125150,-1.024964,-1.179065,-0.952858,-1.101824,...,0.935691,0.875113,-1.066585,-1.066562,-1.061070,-1.093402,-1.082201,-1.106936,-1.122686,-1.183526
31,1.034438,0,1.034438,-0.095459,-0.146784,-0.253657,-0.137775,-0.292313,-0.056766,-0.215196,...,-0.072774,-0.037547,-0.270722,-0.391704,-0.537589,-0.722054,-0.325707,-0.455410,-0.600853,-0.565446
33,-0.009451,0,-0.009451,-2.201586,-2.069055,-1.874077,-2.166044,-2.005172,-2.208692,-2.098121,...,2.864432,3.296602,-2.001577,-1.932022,-1.864693,-1.823926,-1.952460,-1.922690,-1.865519,-1.882488


# Optional - Save Dataframe

In [73]:
merged_df.to_csv(os.path.join(out_dir, "dataframe_for_mixed_effects.csv"))
merged_df.head(3)

,Standardized_Age,One_Hot_Disease,outcome,var_93491,var_93492,var_93493,var_93582,var_93583,var_93672,var_93673,...,var_798109,var_798110,var_798190,var_798191,var_798192,var_798193,var_798281,var_798282,var_798283,var_798373
subject_id,,,,,,,,,,,,,,,,,,,,,
101,0.289341,1,0.289341,0.565606,0.619741,0.824664,0.289703,0.463782,0.107352,0.211271,...,1.397196,1.314386,-0.642376,-0.477894,-0.313008,-0.107899,-0.644884,-0.488569,-0.393913,-0.643514
102,-0.016378,1,-0.016378,0.569502,0.675135,0.978507,0.285238,0.542427,0.006029,0.135294,...,1.697887,1.731429,-0.680607,-0.529925,-0.293708,0.044063,-0.712084,-0.546895,-0.413581,-0.679440
103,-0.888064,1,-0.888064,1.260100,1.192252,1.231898,0.953457,0.973259,0.772010,0.757829,...,1.321081,1.168334,-0.003070,0.211064,0.451404,0.654128,0.044497,0.286226,0.510745,0.284952


# Run The Mixed Effects Analysis

In [74]:
from calvin_utils.file_utils.print_suppression import HiddenPrints
import gc
import os
from tqdm import tqdm
import statsmodels.regression.mixed_linear_model as sm
from patsy import dmatrices


def extract_predictors_from_formula(formula):
    y, X = dmatrices(formula, pd.DataFrame({'voxel': [0, 1], 'Standardized_Age': [0, 1], 'outcome': [0, 1]}), return_type='dataframe')
    return X.columns.tolist()[0:]

def initialize_results_array(num_voxels, expected_predictors, metrics):
    num_metrics = len(expected_predictors) * len(metrics) + 1  # +1 for 'voxel'
    return np.zeros((num_voxels, num_metrics))

def voxelwise_mixed_effects_regression_updated(data_df, formula_template, random_effects_column, model_type='linear', batch_size=50000, checkpoint_path='checkpoint.parquet', use_checkpoints=False):
    """
    Perform mixed-effects regression voxelwise based on the provided formula template.
    
    Parameters:
        data_df (pd.DataFrame): DataFrame containing outcome, voxel values, clinical covariates, and other variables with subjects in rows.
        formula_template (str): A string template for the regression formula with 'voxel' as a placeholder for voxel columns.
        voxel_columns (list): List of voxel column names in data_df.
        random_effects_column (str): The column in data_df to be used for random effects.
        model_type (str, default='linear'): Specifies the type of regression model to use ('linear' or 'logistic').
        batch_size (int, default=5000): Number of voxels to process before saving a checkpoint.
        checkpoint_path (str, default='checkpoint.parquet'): Path to save the intermediate results as a checkpoint.
        use_checkpoints (bool, default=False): whether or not to use checkpoint function

    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values, coefficient values, t-values for each voxel,
                                   along with the coefficient, t-value, and p-value for each predictor.
    """
    # Extract predictors and initialize results array
    voxel_columns = data_df.columns[data_df.columns.get_loc('outcome')+1:]
    expected_predictors = extract_predictors_from_formula(formula_template)
    metrics = ['_coeff', '_t_value', '_p_value']
    num_metrics = len(expected_predictors) * len(metrics) + 1  # +1 for 'voxel'
    num_voxels = len(voxel_columns)
    results_array = np.zeros((num_voxels, num_metrics))
    
    # Existing checkpointing logic
    try:
        if (os.path.exists(checkpoint_path)) & (use_checkpoints):
            results_df = pd.read_parquet(checkpoint_path)
            start_idx = len(results_df)
        else:
            start_idx = 0
    except Exception as e:
        print(f"Failed due to error: {e}.")
    
    # Loop through each voxel column and fit the model
    for idx, voxel in enumerate(tqdm(voxel_columns[start_idx:])):
        formula = formula_template.replace('voxel', voxel)
        
        # Existing mixed-effects logic
        try:
            if model_type == 'linear':
                with HiddenPrints():
                    model = sm.MixedLM.from_formula(formula, data=data_df, groups=data_df[random_effects_column]).fit(method="cg")
            else:
                raise ValueError(f"Unsupported model_type: {model_type}")

            # New: Directly populate the results_array
            col_idx = 1
            for predictor in model.params.index:
                results_array[idx, col_idx:col_idx + 3] = [model.params[predictor], model.tvalues[predictor], model.pvalues[predictor]]
                col_idx += 3

        except Exception as e:
            if str(e) == "Singular matrix":
                pass  # Handle singular matrix cases as needed
            
        # Existing checkpointing logic
        try:
            if ((idx + 1) % batch_size == 0) & (use_checkpoints):
                pd.DataFrame(results_array).to_parquet(checkpoint_path)
                gc.collect()
        except Exception as e:
            print(f"Failed to save checkpoint due to error: {e}.")
    
    # Generate DataFrame from results_array
    column_names = ['voxel'] + [f"{pred}{met}" for pred in expected_predictors for met in metrics]
    results_df = pd.DataFrame(results_array, columns=column_names)
    
    return results_df, model

In [75]:
merged_df.columns

Index(['Standardized_Age', 'One_Hot_Disease', 'outcome', 'var_93491',
       'var_93492', 'var_93493', 'var_93582', 'var_93583', 'var_93672',
       'var_93673',
       ...
       'var_798109', 'var_798110', 'var_798190', 'var_798191', 'var_798192',
       'var_798193', 'var_798281', 'var_798282', 'var_798283', 'var_798373'],
      dtype='object', length=225225)

In [78]:
formula = 'outcome ~ Standardized_Age + voxel  + Standardized_Age:voxel'
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    results_df, model = voxelwise_mixed_effects_regression_updated(merged_df.iloc[:, 0:20], 
                                                formula_template=formula, 
                                                random_effects_column = 'One_Hot_Disease', 
                                                model_type='linear', 
                                                batch_size=100000, 
                                                checkpoint_path='checkpoint.parquet')

100%|██████████| 17/17 [00:01<00:00, 13.25it/s]


View Results

In [79]:
results_df

,voxel,Intercept_coeff,Intercept_t_value,Intercept_p_value,Standardized_Age_coeff,Standardized_Age_t_value,Standardized_Age_p_value,voxel_coeff,voxel_t_value,voxel_p_value,Standardized_Age:voxel_coeff,Standardized_Age:voxel_t_value,Standardized_Age:voxel_p_value
0,0.0,-3.921451e-16,NaN,NaN,1.0,2.598418e+16,0.0,1.053377e-16,3.004563,0.002660,-2.099588e-16,-6.195576,5.807241e-10
1,0.0,-3.489732e-16,NaN,NaN,1.0,3.237543e+16,0.0,1.769894e-17,0.613746,0.539383,-9.852412e-17,-3.651251,2.609658e-04
2,0.0,-4.182281e-16,NaN,NaN,1.0,1.590034e+16,0.0,2.178015e-17,0.369529,0.711734,-2.770315e-16,-5.219780,1.791356e-07
3,0.0,-3.534702e-16,NaN,NaN,1.0,2.911630e+16,0.0,-3.536976e-17,-1.185330,0.235887,-1.230515e-16,-3.923712,8.719512e-05
4,0.0,-3.489732e-16,NaN,NaN,1.0,3.094107e+16,0.0,4.349858e-17,1.506583,0.131918,-1.011909e-16,-3.558411,3.731058e-04
5,0.0,3.651074e-16,NaN,NaN,1.0,1.779975e+16,0.0,7.462789e-18,0.160542,0.872454,4.358717e-16,8.134313,4.142808e-16
6,0.0,-3.552691e-16,NaN,NaN,1.0,3.019603e+16,0.0,1.892359e-17,0.678699,0.497329,-1.305990e-16,-4.223902,2.401084e-05
7,0.0,-3.516714e-16,NaN,NaN,1.0,3.023697e+16,0.0,4.248015e-17,1.477388,0.139572,-1.072869e-16,-3.658131,2.540608e-04
8,0.0,2.446410e-16,NaN,NaN,1.0,3.169312e+16,0.0,-1.820752e-17,-0.695009,0.487050,1.367060e-16,4.598058,4.264479e-06
9,0.0,-9.173659e-17,NaN,NaN,1.0,7.148367e+16,0.0,5.343460e-17,3.811183,0.000138,4.284405e-17,3.339328,8.398143e-04


# Save the Results to Niftis

In [ ]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from calvin_utils.nifti_utils.matrix_utilities import unmask_matrix

def save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None):
    """
    Save each column in the results DataFrame as a NIFTI file.
    
    Parameters:
        results_df (pd.DataFrame): DataFrame containing various statistical measures for each voxel.
        out_dir (str): Directory where NIFTI files should be saved.
        mask_path (str, optional): Path to the NIFTI mask file to use for unmasking.
        mask_threshold (float, optional): Mask threshold for unmasking.
        unmask_by (str, optional): Direction for unmasking ('rows' or 'columns').
        dataframe_to_unmask_by (pd.DataFrame, optional): DataFrame to use for unmasking.
        
    Returns:
        None
    """
    
    # Ensure the output directory exists
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    # Iterate over every column in results_df and generate NIFTI files
    for colname in results_df.columns:
        # Unmask the matrix
        unmasked_df = unmask_matrix(results_df[colname], mask_path=mask_path, mask_threshold=mask_threshold,
                                    unmask_by=unmask_by, dataframe_to_unmask_by=dataframe_to_unmask_by)
        
        # Save the unmasked matrix as a NIFTI file
        view_and_save_nifti(unmasked_df, out_dir, output_name=colname)

In [ ]:
save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None)

(902629,)
[ 93491  93492  93493 ... 798282 798283 798373]


ValueError: shape mismatch: value array of shape (7,) could not be broadcast to indexing result of shape (225222,)

Enjoy

-Calvin